In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re


Download data from kaggle

In [ ]:
#!mkdir ~/.kaggle
#!cp kaggle.json ~/.kaggle/
#!chmod 600 ~/.kaggle/kaggle.json
#!kaggle datasets download -d lakshmi25npathi/online-retail-dataset

Extract Data

In [12]:
#!unzip online-retail-dataset.zip
df=pd.read_excel("online_retail.xlsx")

In [ ]:
df
# df.info()
# df.describe()

Data type casting

In [15]:
def auto_data_type(df):
    for col in df.select_dtypes(include=[np.number]).columns:
        min_value = df[col].min()
        max_value = df[col].max()

        if pd.api.types.is_float_dtype(df[col]):
            df[col] = df[col].astype(np.float32)
        elif min_value >= 0:
            if max_value <= 255:
                df[col] = df[col].astype(np.uint8)
            elif max_value <= 65535:
                df[col] = df[col].astype(np.uint16)
            elif max_value <= 4294967295:
                df[col] = df[col].astype(np.uint32)
            else:
                df[col] = df[col].astype(np.uint64)
        else:
            if -128 <= min_value and max_value <= 127:
                df[col] = df[col].astype(np.int8)
            elif -32768 <= min_value and max_value <= 32767:
                df[col] = df[col].astype(np.int16)
            elif -2147483648 <= min_value and max_value <= 2147483647:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)
    return df


clean_df=auto_data_type(df)
#df.info()


drop rows with missing Customer ID

In [ ]:
df = df.dropna(subset=["Customer ID"])
df["Customer ID"] = df["Customer ID"].astype(np.uint16)

Remove rows with duplicate entries, missing and negative Quantity

In [ ]:
df_copy=df.copy()
df_copy = df_copy.drop_duplicates()
df_copy = df_copy[df_copy["Quantity"] >= 0]
df_copy.info()


Encode Country column to numeric labels

In [18]:
le = LabelEncoder()
df_copy["Country_Code"] = le.fit_transform(df_copy["Country"].astype(str)).astype(np.uint8)

In [19]:
df_copy = df_copy.drop(columns=["Country"])

In [29]:
main_df=df_copy
main_df["Description"] = main_df["Description"].str.lower()
main_df["Description"] = main_df["Description"].str.replace(",", " ")
main_df["Description"] = main_df["Description"].str.replace(r"\s+", "_", regex=True)

invoice_df = main_df.groupby("Invoice")["Description"].apply(" ".join).reset_index()
invoice_df.columns = ["Invoice", "CombinedDescription"]
invoice_df["CombinedDescription"] = invoice_df["CombinedDescription"].str.replace(r"\s+", "_", regex=True)